# Measuring behaviour from wearable cameras and accelerometers

In this practical, you will learn how to download, process and annotate the camera data you have collected. You will also have the opportunity to use models developed by the Oxford Wearables Group to extract activity profiles from your own accelerometer data.

Labelled free-living data is essential for training activity recognition models. By going through the process of annotating your own data, you will begin to understand some of the difficulties of attempting to reduce real life down to a small set of labels.

# 0. Requirements
Before starting this tutorial, we expect you to have the following installed:
- A recent version of Python (≥ 3.10)
- A recent version of Java (OpenJDK) (≥ 8)
- The following Python packages:
    - numpy
    - matplotlib
    - jupyter notebook
    - [pillow](https://pillow.readthedocs.io/en/stable/) (used for image processing) 
    - [actipy](https://pypi.org/project/actipy/) (used for processing accelerometer data)
    
We went through installing these in the previous practical, but in case you are having trouble here is a reminder:

### Reminder
1. Download & install [Miniconda](https://docs.conda.io/en/latest/miniconda.html) (light-weight version of Anaconda).
2. (Mac) Open the terminal. (Windows) Launch the **Miniconda Prompt**.
3. Create a virtual environment:
    ```shell
    conda create -n wearables_practicals python=3.10 openjdk
    ```
    This creates a virtual environment called `wearables_practicals` with Python version 3.10 and Java OpenJDK. 
4. Activate the environment and install the required packages using pip:
    ```shell
    conda activate wearables_practicals
    pip install numpy matplotlib notebook pillow actipy
    ```

*** Alternatively, to install java for actipy on the iMacs, you can use homebrew:
```shell
brew install java
sudo ln -sfn /opt/homebrew/opt/openjdk/libexec/openjdk.jdk /Library/Java/JavaVirtualMachines/openjdk.jdk
```

In [ ]:
# Import required libraries and local scripts
import sys
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import actipy  # for reading in accelerometer data

from PIL import Image

sys.path.append("../scripts")
# Local scripts
import autographer
from sensorplot import ImageData, TextData, ScalarData, VectorData, SensorPlot
from annotate import notebook_annotation

# 1. Prepare camera and accelerometer data

## 1.1 Camera processing

Plug the Autographer wearable camera into the computer using the provided USB cable and make a note of its directory (e.g., `D:/` for Windows, and `/Volumes/` for Mac). If you are having trouble connecting the device or have chosen not to record your own data, please ask your tutors to provide you with pre-recorded data instead.

Check that the Autographer pops up, i.e. no "No such file or directory" errors.

In [ ]:
!ls /Volumes/Autographer/DATA

Download the camera data to your computer. The images will be saved in the directory where you initially cloned the project, under "raw_data/camera".

In [ ]:
# Download the data
autographer.downloadData("/Volumes/Autographer/", "raw_data/camera/")

If the above was successful, you should uncomment and run the command below to erase your data from the camera.

In [ ]:
# autographer.deleteCameraData("/Volumes/Autographer/") # uncomment to run

Execute the following cell to visualise some of the captured images. Select how many images to display by changing the "n_imgs_to_show" variable, and specify how many rows and columns you'd like them to be displayed in with the variables "n_rows" and "n_cols".

In [ ]:
# Change display settings
n_imgs_to_show = 10
n_rows = 2
n_cols = 5

# Prepare a list of image paths along with their timestamps
time_format = "%Y%m%d_%H%M%S"


def get_img_times(paths):
    return [datetime.strptime(path.parts[-1][17:32], time_format) for path in paths]


small_img_paths = list(Path("../raw_data/camera/small").glob("*.JPG"))
small_img_times = get_img_times(small_img_paths)

tuples = list(zip(small_img_paths, small_img_times))
tuples.sort(key=lambda x: x[0])

small_img_paths, small_img_times = zip(*tuples)

# Plot images with timestamps
plt.figure(figsize=(20, 10))

for i, (img_path, img_time) in enumerate(
    zip(small_img_paths[:n_imgs_to_show], small_img_times[:n_imgs_to_show]), 1
):
    plt.subplot(n_rows, n_cols, i)
    plt.imshow(Image.open(img_path))
    plt.title(img_time.strftime("%Hh%Mm%Ss"))
    plt.axis("off")

plt.tight_layout()
plt.show()

# Exercise: estimating camera coverage
The `small_img_times` array contains the times that the images were captured at. 
Can you calculate:
- How many images were captured in total?
- When did the camera start capturing images, and when did it stop?
- What is the average difference in time between consecutive images, and what is the average frame rate?

In [ ]:
# Calc. total number of images
total_n_imgs = ...
print(f"Total number of images: {total_n_imgs}")

# Start and stop time of images
start_time = ...
stop_time = ...
print(f"Start time: {start_time}, stop time: {stop_time}")

# Calc. mean time intervals between images
time_intervals = ...
mean_time_interval = ...
mean_frame_rate = ...
print(f"Mean time interval: {mean_time_interval}, mean frame rate: {mean_frame_rate}")

## 1.2 Accelerometer processing

Plug the Axivity AX3 accelerometer into the computer using the provided USB cable. Again, please check that the accelerometer pops up and take note of its name (it will be something like AX317_43923 under "/Volumes").

In [ ]:
!ls /Volumes

Copy the .CWA file from the accelerometer to the computer in the "raw_data/accelerometer" sub-directory. To do this you can write the correct accelerometer path below, uncomment and then execute the cell.

In [ ]:
%%bash

# mv /Volumes/[INSERT_ACCELEROMETER_ID_HERE]/CWA-DATA.CWA ../raw_data/accelerometer/CWA-DATA.CWA 

Use actipy to read and process the accelerometer data. If you are interested to find out more about the precise processing steps used, you can read the following paper [Large Scale Population Assessment of Physical Activity Using Wrist Worn Accelerometers: The UK Biobank Study (Doherty et al., 2017)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0169649). 

In [ ]:
# Accelerometer reading in
ax3_data, info = actipy.read_device(
    "../raw_data/accelerometer/CWA-DATA.CWA",
    lowpass_hz=20,
    calibrate_gravity=True,
    detect_nonwear=True,
    resample_hz=30,
)

# Plot the accelerometer data
ax3_data[["x", "y", "z"]].plot()
plt.show()

## 1.3 Visualise the camera and accelerometer data together

Convert both sensor data into numpy arrays to allow subsequent plotting.

In [ ]:
# Camera
image_datetimes = np.array(small_img_times, dtype=np.datetime64)
image_paths = np.array(small_img_paths)

# Accelerometer
sensor_datetimes = ax3_data.index.to_numpy()
accelerometer_readings = ax3_data[["x", "y", "z"]].to_numpy()
light_readings = ax3_data["light"].to_numpy()
temperature_readings = ax3_data["temperature"].to_numpy()

Select an appropriate time window to display by specifying which image to start from ("start_time" variable) and how many minutes/seconds to display ("duration" variable).

In [ ]:
# Prepare data for plotting by creating a list of SensorData objects
sensor_data = [
    ImageData("Camera", image_datetimes, image_paths, plot_x_ticks=True, img_zoom=0.22),
    ScalarData(
        "Temperature", sensor_datetimes, temperature_readings, plot_x_ticks=False
    ),
    ScalarData("Light", sensor_datetimes, light_readings, plot_x_ticks=False),
    VectorData(
        "Accelerometer",
        sensor_datetimes,
        accelerometer_readings,
        plot_x_ticks=10,
        dim_names=["x", "y", "z"],
    ),
]

sv = SensorPlot(sensor_data)
print(sv)

In [ ]:
# Select time window to plot
start_time = image_datetimes[
    50
]  # change this to a sensible start time, e.g. image_datetimes[0]
duration = np.timedelta64(
    30, "s"
)  # change this to a sensible duration, e.g. 5 minutes, i.e. np.timedelta64(5, 'm')

print(
    f"Looking at data from {str(start_time)[11:19]} to {str(start_time + duration)[11:19] }"
)

# Plot the data
fig, ax = sv.plot_window(start_time, duration)
plt.show()

# 2. Annotate the image data

In order to annotate each image taken by the camera, we need a set of annotations to choose from. This set of possible annotations is called the annotation schema. For detailed annotations of physical activity, we tend to use the [compendium of physical activity](https://sites.google.com/site/compendiumofphysicalactivities/Activity-Categories?authuser=0) to inform our annotation schema.

We've put together a simple function using maptlotlib to allow you to label the image data inline in this jupyter notebook.
This is implemented in the `notebook_annotation` function.    
    
The following commands are used to navigate:
- `next`/`.` - move to the next N images (if there are any left, but only jumping one image along)
- `prev`/`,` - move to the previous N images (if there are any left, but only jumping one image along)
- `copy`/`c` - copy the current annotation to the next image, and display the next N images
- `quit`/`q` - quit the loop, saving the annotations to the numpy array

A particularly useful shortcut for quickly annotating the same activity multiple times is:
- `copy N`, or `c N`, where `N` is an integer. This copies the last annotations to the next `N` images. 

To make annotation faster, you can define shortcuts for each label, so that you can just enter the shortcut as opposed to the whole label.

Importanty, you can choose to proceed with the example annotation schema provided below (Sedentary behaviour, Light physical activity, Moderate-to-vigorous physical activity), or come up with your own.

In [ ]:
label_dir_name = (
    "../raw_data/annotations/activites"  # path to where annotations will be saved
)

schema = {  # come up with a better schema
    # shortcut: long name
    "s": "Sedentary",
    "l": "Light",
    "m": "MVPA",
}

notebook_annotation(
    label_dir_name,  # Where to save the annotations
    schema,  # The schema to annotate your data with
    image_paths,
    image_datetimes,
    imgs_to_display=5,  # How many images to display at once
    save_freq=10,  # How often to save the annotations
    figsize=(30, 10),  # This can be made bigger
)

In [ ]:
# Reload the saved annotations and add it to the SensorPlot
annotations = np.load(label_dir_name + "/labels.npy", allow_pickle=True)
sv.add_data(
    TextData(
        "Annotations", image_datetimes, annotations, plot_x_ticks=False, fontsize=10
    ),
    index=1,
    height_ratio=0.3,
)
print(sv)

In [ ]:
# Select time window to plot
start_time = image_datetimes[
    0
]  # change this to a sensible start time, e.g. image_datetimes[10]
duration = np.timedelta64(
    60, "s"
)  # change this to a sensible duration, e.g. 5 minutes, i.e. np.timedelta64(5, 'm')

print(
    f"Looking at data from {str(start_time)[11:19]} to {str(start_time + duration)[11:19] }"
)

# Plot the data
fig, ax = sv.plot_window(start_time, duration)
plt.show()

## Questions to discuss 
1) Think about what makes a good annotation schema. Should each image be uniquely described by a single label, or should multiple labels apply to each image?

2) What are the advantages and disadvantages of choosing a detailed annotation schema with many labels?

3) How can we deal with bias introduced by the annotator, including biases that arise from practical issues such as fatigue from annotating many images?

## Excercise: Run pre-trained models on your own data, to classify your activity epochs

Go to the [biobankAccelerometerAnalysis](https://github.com/OxWearables/biobankAccelerometerAnalysis) GitHub page and follow the installation and usage instructions carefully.

Run the tool to produce summary movement statistics from your Axivity file (.cwa). How do the model predictions (acc_id-timeSeries.csv file) compare to your own annotations?